In [1]:
import numpy as np
import time
import psutil

# --- Benchmark controls ---
MAX_TOKENS = 128

# Get the EOS token id from the loaded model
EOS_ID = llm.token_eos()
print("EOS token id:", EOS_ID)

def ban_eos_logits_processor(input_ids, logits):
    """
    llama-cpp-python logits_processor hook:
    - input_ids: array of token ids so far
    - logits: array of logits for next token
    We set EOS logit to -inf so generation won't end early.
    """
    # logits can be numpy array; mutate in place
    logits[EOS_ID] = -1e10
    return logits

NameError: name 'llm' is not defined

In [2]:
prompt = (
    "Write a continuous technical explanation of energy efficiency in AI inference. "
    "Do not conclude. Keep expanding with details, examples, and tradeoffs.\n\n"
)

In [3]:
def run_once():
    process = psutil.Process()
    mem_before = process.memory_info().rss / 1e6

    t0 = time.time()
    out = llm(
        prompt,
        max_tokens=MAX_TOKENS,
        temperature=0.0,     # deterministic
        top_p=1.0,           # disable nucleus sampling
        top_k=0,             # no top-k restriction
        repeat_penalty=1.0,  # keep neutral for benchmarking
        logits_processor=[ban_eos_logits_processor],
    )
    t1 = time.time()

    mem_after = process.memory_info().rss / 1e6

    text = out["choices"][0]["text"]
    n = out["usage"]["completion_tokens"]
    elapsed = t1 - t0
    tps = n / elapsed if elapsed > 0 else float("inf")

    return {
        "tokens": n,
        "seconds": elapsed,
        "toks_per_sec": tps,
        "mem_delta_mb": mem_after - mem_before,
        "text_tail": text[-120:],  # last bit, useful for debugging
    }


In [4]:
results = []
for i in range(3):
    r = run_once()
    results.append(r)
    print(f"Run {i+1}: tokens={r['tokens']}  time={r['seconds']:.2f}s  tok/s={r['toks_per_sec']:.2f}")

print("\nToken counts:", [r["tokens"] for r in results])


NameError: name 'llm' is not defined